In [ ]:
import numpy as np
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
from torch.optim import lr_scheduler
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
exercise_mapping = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat': 5,
    'Stretch': 6
}
set_mapping = {
    'Correct' : 0,
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6
}

In [ ]:
#Importing pre dataset
train_df = pd.read_parquet('/content/drive/MyDrive/ML_project/RNN/train_set_not_std_less_noise.parquet', engine='pyarrow')
test_df = pd.read_parquet('/content/drive/MyDrive/ML_project/test_set_not_std.parquet', engine='pyarrow')

In [ ]:
video_indices = test_df.groupby('video_id').size().values
video_indices = np.insert(video_indices, 0, 0)
video_indices = np.cumsum(video_indices)

In [ ]:
get_mlp3x256 = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(100, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

In [ ]:
get_physio_mlp = lambda: torch.nn.Sequential(

    nn.Flatten(),
    nn.Linear(107, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512, 7)
)

# Training the model to predict exercises
We first split the train and test set


In [ ]:
X_train, X_test, y_train, y_test = helpers.import_data_exercise(train_df, test_df, exercise_mapping, True)

We then define everything we need to train our model

In [ ]:
#Defines hyperparameters
learning_rate = 2e-3
batch_size = 128

# Creates an instance of the dataset
dataset = TensorDataset(X_train, y_train)

# Creates a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Defines the model
model = get_mlp3x256_ex()
model.train()

# Defines the path to save the model's state dictionnary
model_path = '/content/drive/MyDrive/ML_project/MLPEX.path'

# Defines loss function
criterion = nn.CrossEntropyLoss()

# Creates an optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Creates a scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

We procede to train the model. It converges to sufficiently trained state to perform 100% accuracy on test set in a single iteration. Note that the already trained model state dictionnary can be loaded from the path 'MLPEX_trained.path' by uncommenting the cell below.

In [ ]:
model.load_state_dict(torch.load('MLPEX_trained.path'))

In [ ]:
# Trains the model
helpers.train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path)

# Prints a report on the accuracy and F1 score
helpers.test_accuracy(model, X_test,y_test,video_indices)

Processing: 2.38%

KeyboardInterrupt: ignored

#Training of MLP 3x256 to predict sets

In [ ]:
X_train, X_test, y_train, y_test = import_data_set(train_df, test_df, set_mapping, True)

In [ ]:
#Defines hyperparameters
learning_rate = 0.001
batch_size = 128

# Creates an instance of the dataset
dataset = TensorDataset(X_train, y_train)

# Creates a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Defines the model
model = get_mlp3x256_set()
model.train()

# Defines the path to save the model's state dictionnary
model_path = '/content/drive/MyDrive/ML_project/PhysioMLP.path'

# Defines loss function
criterion = nn.CrossEntropyLoss()

# Creates an optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Creates a scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)

In [ ]:
for i in range(30):
  # Trains the model
  train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path, batch_size)

  # Prints a report on the accuracy and F1 score
  test_accuracy(model, X_test,y_test,video_indices)

Processing: 100.00%Epoch [1/1], Loss: 1.2220
Accuracy on each frame: 0.4192
Accuracy for videos: 0.5190
Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.83      0.60       140
           1       0.58      0.34      0.43        76
           2       0.71      0.35      0.47        72
           3       0.50      0.39      0.44        64
           4       0.78      0.45      0.57        40
           5       0.60      0.38      0.46        16
           6       0.22      0.17      0.19        12

    accuracy                           0.52       420
   macro avg       0.55      0.41      0.45       420
weighted avg       0.56      0.52      0.50       420

Processing: 100.00%Epoch [1/1], Loss: 1.2193
Accuracy on each frame: 0.4184
Accuracy for videos: 0.5238
Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.84      0.60       140
           1       0.58      0.34      0